# Calculate zonal nighttime lights and animated nighttime lights

This notebook calculates zonal statistics on VIIRS nighttime lights for a series of industrial parks. For each of those parks, we calculate a series of buffers, and summarize nighttime lights.

Finally, and animation of nighttime lights is created for a 10km buffer around the IP.

In [1]:
import sys, os, importlib
import rasterio

import pandas as pd
import geopandas as gpd

from shapely.geometry import Point, box

sys.path.append("../../../../gostrocks/src")

import GOSTRocks.rasterMisc as rMisc
from GOSTRocks.misc import tPrint


/home/wb411133/.conda/envs/geog/lib/python3.7/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.7.1-CAPI-1.11.1 0) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
ip_locations = "Data/IP_Locations.csv"
out_folder = "/home/wb411133/temp/ETH_NTL/" 
viirs_folder = os.path.join(out_folder, "Data/VIIRS")
zonal_res = os.path.join(out_folder, "Data/IP_Locations_NTL_ZONAL.csv")

if not os.path.exists(viirs_folder):
    os.makedirs(viirs_folder)
    
# Read in csv, re project to UTM, and write to file
inD = pd.read_csv(ip_locations)
inD_geom = [Point(x['Lon'], x['Lat']) for idx, x in inD.iterrows()]
inD = gpd.GeoDataFrame(inD, geometry=inD_geom, crs="epsg:4326")
inD = inD.to_crs("EPSG:20138")
inD.to_file(ip_locations.replace(".csv", ".geojson"), driver="GeoJSON")

In [14]:
inD.head()

,Industrial Park,Region,Location,Lat,Lon,Zone,Public/Private,Operational,geometry,BUFFER_KM,...,viirs_201901,viirs_201902,viirs_201903,viirs_201904,viirs_201905,viirs_201906,viirs_201907,viirs_201908,viirs_201911,viirs_201912
0,Adama IP,Oromia,"8.513708, 39.305190",8.513708,39.30519,East Shewa Zone,Public,Yes,"POLYGON ((39.35040 8.51437, 39.35024 8.50996, ...",5,...,968.739990,1045.190063,910.320007,878.489990,1042.099976,965.509949,794.030029,876.860046,975.869995,1000.630005
0,Adama IP,Oromia,"8.513708, 39.305190",8.513708,39.30519,East Shewa Zone,Public,Yes,"POLYGON ((39.39561 8.51503, 39.39530 8.50620, ...",10,...,2326.189941,2626.120117,2256.609863,2286.050049,2685.000000,2419.290039,2071.359863,2226.799805,2512.830078,2507.459961
0,Adama IP,Oromia,"8.513708, 39.305190",8.513708,39.30519,East Shewa Zone,Public,Yes,"POLYGON ((39.44082 8.51568, 39.44036 8.50243, ...",15,...,2707.350098,3330.069824,2888.270020,3042.719971,3331.429932,3069.679932,2593.250000,2808.219971,3283.680176,3105.339844
0,Adama IP,Oromia,"8.513708, 39.305190",8.513708,39.30519,East Shewa Zone,Public,Yes,"POLYGON ((39.48604 8.51632, 39.48542 8.49866, ...",20,...,3253.020020,4327.069824,3755.219971,4145.109863,4394.709961,4013.719727,3484.209961,3719.080078,4495.660156,4062.929932
0,Adama IP,Oromia,"8.513708, 39.305190",8.513708,39.30519,East Shewa Zone,Public,Yes,"POLYGON ((39.53126 8.51696, 39.53049 8.49489, ...",25,...,3894.729736,5549.410156,4822.810059,5424.929688,5547.620117,4937.229980,4352.669434,4733.490234,5814.850586,5075.049805


In [16]:
inD['Industrial Park'].unique()

array(['Adama IP', 'Bahir Dar IP', 'Bole Lemi IP', 'CCCC Arerti', 'DBL',
       'Debre Birhan', 'Dire Dawa', 'Eastern Industrial Zone',
       'Hawassa IP', 'Huajian IP', 'ICT IP', 'Jimma IP',
       'Kilinto Pharmaceutical IP', 'Kombolcha IP', 'Mekele IP',
       'Mojo George Shoe Industrial Zone', 'Vogue/Velocity'], dtype=object)

# Zonal Stats

In [3]:
buffers = range(5, 51, 5)
try:
    del final
except:
    pass

for buf in buffers:    
    curD = inD.copy()
    curD['geometry'] = curD.buffer(buf * 1000)
    curD['BUFFER_KM'] = buf
    try:
        final = final.append(curD)
    except:
        final = curD
final = final.sort_values(["Industrial Park", "BUFFER_KM"])
inD = final
inD = inD.to_crs("epsg:4326")
inD.to_file(ip_locations.replace(".csv", "_BUFFERED.geojson"), driver="GeoJSON")

In [4]:
# Get a list of the VIIRS images in S3. This example leverages the GOST teams S3 bucket
s3_base = 's3://wbgdecinternal-ntl/'
ntl_file_list = "/home/wb411133/temp/YEM/AWS_NTL_S3.txt"
focal_tile = "TILE2"
temp_folder = "/home/wb411133/temp"
viirs_folder = "/home/wb411133/temp/ETH/Hawassa/VIIRS/"
if not os.path.exists(viirs_folder):
    os.makedirs(viirs_folder)

out_map_folder = "/home/wb411133/temp/ETH/Hawassa/VIIRS_MAPS"
if not os.path.exists(out_map_folder):
    os.makedirs(out_map_folder)


all_files = []
with open(ntl_file_list, 'r') as in_aws:
    for line in in_aws:
        if focal_tile in line and 'avg_rade9' in line:
            all_files.append(os.path.join(s3_base, line.split(" ")[-1][:-1]))
            
all_files[:5]

['s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201204/TILE2/SVDNB_npp_20120401-20120430_75N060W_vcmcfg_v10_c201605121456.avg_rade9h.tif',
 's3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201205/TILE2/SVDNB_npp_20120501-20120531_75N060W_vcmcfg_v10_c201605121458.avg_rade9h.tif',
 's3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201206/TILE2/SVDNB_npp_20120601-20120630_75N060W_vcmcfg_v10_c201605121459.avg_rade9h.tif',
 's3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201207/TILE2/SVDNB_npp_20120701-20120731_75N060W_vcmcfg_v10_c201605121509.avg_rade9.tif',
 's3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201208/TILE2/SVDNB_npp_20120801-20120831_75N060W_vcmcfg_v10_c201602121348.avg_rade9.tif']

In [12]:
# Run zonal statistics against the define admin (in_zones)
for cur_tif in all_files:
    res = rMisc.zonalStats(inD, cur_tif, minVal=0.05)
    res = pd.DataFrame(res,columns=['SUM','MIN','MAX','MEAN'])
    inD[cur_tif.split("/")[5]] = res['SUM'].values
    tPrint(cur_tif)
    
pd.DataFrame(inD.drop(['geometry'], axis=1)).to_csv(zonal_res)

13:30:08	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201204/TILE2/SVDNB_npp_20120401-20120430_75N060W_vcmcfg_v10_c201605121456.avg_rade9h.tif
13:30:13	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201205/TILE2/SVDNB_npp_20120501-20120531_75N060W_vcmcfg_v10_c201605121458.avg_rade9h.tif
13:30:17	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201206/TILE2/SVDNB_npp_20120601-20120630_75N060W_vcmcfg_v10_c201605121459.avg_rade9h.tif
13:30:27	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201207/TILE2/SVDNB_npp_20120701-20120731_75N060W_vcmcfg_v10_c201605121509.avg_rade9.tif
13:30:37	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201208/TILE2/SVDNB_npp_20120801-20120831_75N060W_vcmcfg_v10_c201602121348.avg_rade9.tif
13:30:48	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201210/TILE2/SVDNB_npp_20121001-20121031_75N060W_vcmcfg_v10_c201602051401.avg_rade9.tif
13:30:58	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201211/TILE2/SVDNB_npp_20121101-20121130_75N060W_vcmcfg_v10_c201601270845.avg_rade9.

13:39:45	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201704/TILE2/SVDNB_npp_20170401-20170430_75N060W_vcmcfg_v10_c201705011300.avg_rade9.tif
13:39:54	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201705/TILE2/SVDNB_npp_20170501-20170531_75N060W_vcmcfg_v10_c201706021500.avg_rade9.tif
13:40:04	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201706/TILE2/SVDNB_npp_20170601-20170630_75N060W_vcmcfg_v10_c201707021700.avg_rade9h.tif
13:40:14	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201707/TILE2/SVDNB_npp_20170701-20170731_75N060W_vcmcfg_v10_c201708061230.avg_rade9h.tif
13:40:23	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201708/TILE2/SVDNB_npp_20170801-20170831_75N060W_vcmcfg_v10_c201709051000.avg_rade9h.tif
13:40:32	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201709/TILE2/SVDNB_npp_20170901-20170930_75N060W_vcmcfg_v10_c201710041620.avg_rade9h.tif
13:40:42	s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201710/TILE2/SVDNB_npp_20171001-20171031_75N060W_vcmcfg_v10_c201711021230.avg_rade9

# Generate animated maps

In [ ]:
# If you want to clip out the raster data for the below mapping, run this block
sel_D = inD.loc[inD['BUFFER_KM'] == 10]
for idx, row in sel_D.iterrows():
    tPrint(out_folder)
    
    out_folder = os.path.join(viirs_folder, row['Industrial Park'])
    out_map_folder = os.path.join(out_folder, "MAPS")
    if not os.path.exists(out_map_folder):
        os.makedirs(out_map_folder)
    
    viirs_files = []
    for cur_tif in all_files:
        file = f'{cur_tif.split("/")[5]}.tif'
        out_file = os.path.join(os.path.join(out_folder, "%s" % file))
        viirs_files.append(out_file)
        if not os.path.exists(out_file):
            row['geometry'] = box(*row.geometry.bounds)
            rMisc.clipRaster(rasterio.open(cur_tif), gpd.GeoDataFrame(pd.DataFrame(sel_D.loc[idx]).transpose(), geometry="geometry", crs=inD.crs), out_file)    
            
    viirs_files.sort()
    out_files = []
    for f in viirs_files:
        out_file = os.path.join(out_map_folder, os.path.basename(f))
        out_files.append(out_file)
        rMisc.map_viirs(f, out_file, text_x = 0, text_y = 6)
        
    kwargs = {'duration':0.3}
    images=[]
    for filename in out_files:
        images.append(imageio.imread(filename))

    imageio.mimsave("%s_timelapse.gif" % out_folder, images, **kwargs)
    

In [ ]:
importlib.reload(rMisc)
in_tif = f
rMisc.map_viirs(f, out_file, text_x = 0, text_y = 6)

In [ ]:
rasterio.open(in_tif).read()

In [ ]:
importlib.reload(rMisc)
for idx, row in sel_D.iterrows():
    out_folder = os.path.join(viirs_folder, row['Industrial Park'])
    viirs_files = [os.path.join(out_folder, x) for x in os.listdir(out_folder) if x.endswith("tif")]
    

In [ ]:
import imageio
for filename in out_files:
    images.append(imageio.imread(filename))

imageio.mimsave("%s_timelapse.gif" % out_folder, images, **kwargs)

In [ ]:
"%s_timelapse.gif" % out_folder